In [71]:
%matplotlib inline 
import re
import requests
import numbers
import sqlite3
import numpy as np
import pandas as pd
import missingno as msno # Visualization
import sys
from bs4 import BeautifulSoup

---

In [74]:
db_name = 't_homechef.sqlite'
show_db_tables(db_name)

['meal_ingredients', 'meal_info', 'meal_bow']


In [75]:
# Download the 3 tables unto 3 dataframes: 
meal_info, meal_ing, meal_bow = load_3_dataframes('meal_info', 'meal_ingredients','meal_bow', db_name)

# Download my csv ingredients file unto dataframe: "data"
data = get_my_homechef_csv('my_meals.csv')

# Check for duplicates in both dataframes. 
meal_info, add_new = compare_csv_vs_web(data, meal_info)

Index(['meal_id', 'title', 'rating', 'duration', 'expiration', 'exp_level',
       'spiciness', 'category'],
      dtype='object')
Index(['meal_id', 'Star'], dtype='object')


/Users/iZbra1/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:119: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/Users/iZbra1/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


## Add the new csv-records to a new database. 

In [82]:
# SQL VARIABLES FOR MEAL_INFO TABLE
sqlitefile = 't_csv_meals.sqlite'

table_name           = 'meal_info'  # name of the table to be created
id_field             = 'meal_id'    # name of the ID column
title_col            = 'title'
rating_col           = 'rating'     # integer counting the starts [0,5]
duration_col         = 'duration'   # text for now -> integer counting time in minutes 
expiration_col       = 'expiration' # Integer counting in days
expertise_col        = 'exp_level'  # Text -> Categorical 
spicy_col            = 'spiciness'  # Text -> Categorical
cat_col              = 'category'   # Text -> Categorical

# SQL VARIABLES FOR INGREDIENTS TABLE
table_name_2         = 'meal_ingredients' 
id_column            = 'meal_id'
ing_name             = 'ingredients' # name of the column

field_type_1         = 'INTEGER'
field_type_2         = 'TEXT'  # column data type

meal_count = 0

for newid, news, newm, newt in zip(add_new.Meal_id, add_new.Star, add_new.Main, add_new.Title):
    page_url = 'https://www.homechef.com/'+str(newid)
    m_rating = news
    ctgry = newm
    meal_title = newt
    
    #print('\nScraping',page_url,meal_title,m_rating,ctgry)  
    add_single_meal(sqlitefile, page_url, ctgry, m_rating)

Meal ID 2282 was not added
 UNIQUE constraint failed: meal_info.meal_id
Meal not found
m_id is 0
Meal ID 2349 Added!
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal ID 2859 Added!
Meal not found
m_id is 0
Meal ID 3090 Added!
Meal not found
m_id is 0
Meal ID 3287 Added!
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal ID 3319 Added!
Meal ID 3333 Added!
Meal ID 3336 Added!
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal ID 3422 Added!
Meal ID 3428 Added!
Meal not found
m_id is 0
Meal ID 3561 Added!
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal ID 3692 Added!
Meal ID 3693 Added!
Meal ID 3694 Added!
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal not found
m_id is 0
Meal ID 3762 Added!
Meal ID 3773 Added!
Meal not found
m_id is 0
Mea

## CSV Scraping Functions

In [83]:
def add_single_meal(sqlitefile, page_url, ctgry, m_rating=0):
# Build the new url leading to the single meal page of given category
    meal_url = requests.get(page_url) 
    tryout = False
    
    # LOADED PAGE CHECK
    if meal_url.status_code != 200:
        s = re.sub(',','',meal_title)
        page_url = 'https://www.homechef.com/meals/'+re.sub(' ','-',s).lower() 
        tryout = True
        meal_url = requests.get(page_url)

    if tryout and meal_url.status_code != 200:
        #print('Tried', page_url, 'didnt work either!')
        #continue
        return 0

    meal = BeautifulSoup(meal_url.content,'html.parser')

    # GET SINGLE MEAL ID
    m_id = get_single_meal_id(meal)
    if m_id == 0:
        #print('m_id is',m_id)
        #continue
        return 0
        
    # GET MEAL TITLE
    m_title = meal.select('header.meal__header.order--1.group h1')[0].get_text()
    m_sub = meal.select('header.meal__header.order--1.group h2')[0].get_text()

# GET THE MEAL'S PARAMETERS AND INGREDIENTS:
# ingredients = [qty, mUnit, key_type, key_ing, key2_ing] 
# and overtime = [time_spent, expire, exp_level, spicy]
    ingredients, overview = get_meal_overview_and_ingredients(meal, m_id)

    # DATA CHECK
    if not(ingredients or overview):
        #print('No ingredients info found for', meal_list[indx][1])
        #continue
        return 0

    conn = sqlite3.connect(sqlitefile)
    c = conn.cursor()

# INSERT OVERVIEW RECORD IN MEAL_INFO TABLE THE VALUES 
# Meal Id, Meal Name, Rating, Duration, Expiration Time, Difficulty,Spice level and Category. 
# SQL INSERT QUERY
    ins_query = "INSERT INTO {tn} ({idf}, {tc}, {rc}, {dc}, {ec}, {ex}, {sc}, {ct})\
         VALUES ('{mid}', '{tv}', '{star}', '{ov0}', '{ov1}', '{ov2}', '{ov3}', '{cat}')".\
    format(tn=table_name,   idf=id_field,       tc=title_col,     rc=rating_col, 
           dc=duration_col,  ec=expiration_col, ex=expertise_col, sc=spicy_col, ct=cat_col, 
           mid=m_id, 
           tv=m_title+' '+m_sub, star=m_rating,
           ov0=overview[0], ov1=overview[1], ov2=overview[2], ov3=overview[3], cat=ctgry)

    try: 
        c.execute(ins_query)
        #print('Meal ID', m_id, 'Added!')
        meal_added = True   

    except Exception as oe:
        #print('Meal ID',m_id, 'was not added\n',oe)
        meal_added = False

    # INSERT INGREDIENTS RECORD IN MEAL_INGREDIENTS TABLE
    if meal_added:
        for item in ingredients:
            c.execute("INSERT OR IGNORE INTO {tn} ({idf}, {tc}) VALUES ('{mid}', '{m_ing}')".\
                    format(tn=table_name_2,idf=id_column, tc=ing_name,
                    mid=m_id,m_ing=item)) 
    conn.commit()
    conn.close()
    
# receives the html format of the recipe and returns the id of the meal
def get_single_meal_id(meal):
    m_id = meal.select('a.button.link--inverse.list--share__link')
    if m_id != []:
        s = m_id[0].get('href')
    else: 
        #print('Meal not found')
        return 0
    r = re.search(r"/\d+/",s)
    if r:
        return s[r.start():r.end()].rstrip('/').lstrip('/')
    else: 
        #print('Meal_id not found')
        return np.nan
    
# Prep csv dataset
def get_my_homechef_csv(csvfile):
    data = pd.read_csv(csvfile)
    data = data[data.Company == 'HomeChef']
    # Convert meal_id to int
    data['Meal_id'] = [int(ind) for ind in data.ID]
    # Convert star to string to float. 
    data['s1']=[str(val) if val != val else re.sub(',','.',val) for val in data.Star.values]
    data['Star'] = data.s1.astype(float)
    return data.drop('s1', axis=1)

# Download the 3 tables as 3 dataframes
def load_3_dataframes(table_name_1, table_name_2, table_name_3, db_name = 'project2.sqlite'):
    id_field      = 'meal_id'
    q1 = 'SELECT * FROM {tn} ORDER BY {idf}'.format(tn=table_name_1, idf=id_field)
    q2 = 'SELECT * FROM {tn} ORDER BY {idf}'.format(tn=table_name_2, idf=id_field)
    q3 = 'SELECT * FROM {tn}'.format(tn=table_name_3)
    conn = sqlite3.connect(db_name)
    # Load Meal_info Dataframe
    meal_info = pd.read_sql_query(q1, conn)
    try: 
        meal_info = meal_info.drop('index', axis=1)
    except: 
        pass
    # Load Meal_ing Dataframe
    meal_ing = pd.read_sql_query(q2, conn)
    try:
        meal_ing = meal_ing.drop('index', axis=1)
    except: 
        pass
    # Load Ingredients Dataframe
    t3 = pd.read_sql_query(q3, conn)
    try: 
        t3 = t3.drop('index', axis=1)
    except:
        pass
    conn.close()
    return meal_info, meal_ing, t3

def get_sql_table_columns(db_name,table_name):
    get_col_query = 'select * from {t1}'.format(t1=table_name)
    conn = sqlite3.connect(db_name)
    conn.row_factory = sqlite3.Row
    c = conn.execute(get_col_query)
    row = c.fetchone()
    return row.keys()

def show_db_tables(db_name):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    query_check  = 'SELECT name FROM sqlite_master WHERE type= "table";'
    c.execute(query_check)
    table_set = [item[0] for item in c.fetchall()]
    print(table_set)

def get_meal_overview(meal_ov, idx, w0, w1):
#    print('get_meal_overview')
    while idx < len(meal_ov):
        mov = meal_ov[idx].get_text().lower().split()
        if mov != [] and (mov[0] == w0 and mov[1] == w1):
            return meal_ov[idx+1].get_text().lower().split()[0], idx+1
        idx += 1
    return 0
        
def get_meal_overview_and_ingredients(meal, mId):
    
    # Meal Overview
    meal_ov = meal.select('div.meal__overviewItem span')
    if meal_ov == []: # Some cases where the recipe is so logic that doesnt need info (i.e spring fruit basket)
        return [0,0]

    ## 2 is time, 4 is within days, 6 or 7 is level, 13 or 14 is spicy 18 ends
    time_spent, idx = get_meal_overview(meal_ov, 0, 'cook', 'time:')
    expire,     idx = get_meal_overview(meal_ov, idx, 'cook','within:')
    exp_level,  idx = get_meal_overview(meal_ov, idx,'difficulty', 'level:')
    spicy,      idx = get_meal_overview(meal_ov, idx,'spice', 'level:')
    # Meal Ingredients
    
    meal_ing = meal.select('ul.list--unstyled.group.position--relative.text--center--bpDown2 li')
    ing_list = []
    for ingredient in meal_ing:
        ### example of ingredient: ['info', '3¾', 'oz.', "font'ina", 'cheese', 'slices']        
        ing = ingredient.get_text().lower().replace("info","").replace("'","_").split()
        ing_list.append(' '.join(ing))

    return ing_list, [time_spent, expire, exp_level, spicy]

'''
Compares the records from the csv file and the dataframe meal_info
returns the meal_info with the rating taken from the csv and a list of records 
that are to be added in the meal_info dataframe. 
'''
def compare_csv_vs_web(data, meal_info):
    list_to_scrap = []
    already_in = []
    for m_id in data.Meal_id:
        if int(m_id) in meal_info.meal_id.values:
            already_in.append(m_id)
        else: 
            list_to_scrap.append(m_id)
    add_new = data.loc[data.Meal_id.isin(list_to_scrap),['Meal_id','Title','Star','Main']].sort_values('Meal_id')
    up_rat = data.loc[data.Meal_id.isin(already_in),['Meal_id','Title','Star']].sort_values('Meal_id')
    # Take the rating from the csv and pastes it on the meal_info df'''
    df2 = up_rat.loc[up_rat.Meal_id.isin(already_in),['meal_id','Star']].sort_values('meal_id')
    # Merge Column data.Star to meal_info.Rating
    print(meal_info.columns)
    print(df2.columns)
    m_i = pd.merge(meal_info, df2, on='meal_id', how='outer')
    meal_info = m_i.drop(m_i.index[m_i[m_i.meal_id == 3704].index[0]]) # Removing a duplicate

    meal_info['new_rating'] = [x if y!=y else y for x,y in zip(meal_info.rating, meal_info.Star)]
    meal_info = meal_info.drop(['rating', 'Star'], axis=1)
    meal_info = meal_info.rename(columns={'new_rating': 'rating'})
    return meal_info, add_new

